# Domain_walls

### Estimate polarization

In [ ]:
import numpy as np
from ase.geometry import find_mic

class PolarizationEstimator:
    """
    This script uses the point charge model and you need to have both optimized CONTCAR and high symmtry structure POSCAR files to determine the polarization.
    """
    def __init__(self, centrosymmetric_structure, ferroelectric_structure, nominal_charges=None, cation_types=None, anion_types=None, ifnormalized=True):
        self.cs_struct = centrosymmetric_structure
        self.fe_struct = ferroelectric_structure
        self.cation_types = set(cation_types) if cation_types else set()
        self.anion_types = set(anion_types) if anion_types else set()
        self.nominal_charges = None
        self.ifnormalized = ifnormalized

        # Validate input
        if nominal_charges is not None:
            self.nominal_charges = np.array(nominal_charges)
            if len(self.nominal_charges) != len(self.cs_struct):
                raise ValueError("Length of nominal charges must match the number of atoms.")
        elif self.cation_types and self.anion_types:
            self._assign_charges_based_on_types()
        else:
            raise ValueError("Either nominal_charges or both cation_types and anion_types must be specified.")

        if len(self.cs_struct) != len(self.fe_struct):
            raise ValueError("The number of atoms in both structures must be the same.")

    def _assign_charges_based_on_types(self):
        """
        Assign nominal charges based on cation and anion types.
        """
        symbols = self.cs_struct.get_chemical_symbols()
        charges = []
        for symbol in symbols:
            if symbol in self.cation_types:
                charges.append(1.0)  # Default cation charge
            elif symbol in self.anion_types:
                charges.append(-1.0)  # Default anion charge
            else:
                raise ValueError(f"Element '{symbol}' not found in cation_types or anion_types.")
        self.nominal_charges = np.array(charges)

    def compute_ionic_polarization(self):
        """
        Compute the ionic polarization direction vector (normalized).
        """
        pos_centro = self.cs_struct.get_positions(wrap=True)
        pos_ferro = self.fe_struct.get_positions(wrap=True)
        cell = self.cs_struct.get_cell()

        displacements = find_mic(pos_ferro - pos_centro, cell=cell)[0]
        dipole_contributions = self.nominal_charges[:, None] * displacements

        total_dipole = dipole_contributions.sum(axis=0)
        volume = self.cs_struct.get_volume()
        polarization_vector = total_dipole / volume
        if self.ifnormalized:
            return polarization_vector / np.linalg.norm(polarization_vector)
        else:
            return polarization_vector


In [ ]:
from ase.io import read
ini_atoms = read("../examples/Domain_walls/GeTe/POSCAR")
fin_atoms = read("../examples/Domain_walls/GeTe/CONTCAR")
polarization = PolarizationEstimator(ini_atoms, fin_atoms, cation_types=["Ge"], anion_types=["Te"]).compute_ionic_polarization()
polarization

### Build domain wall
下一步：根据 slab1 顶端基团和 slab2 底端基团判断电荷类型

In [ ]:
from ase import Atoms
from ase.build import cut, rotate
from ase.io import read
from ase.visualize import view
from functools import reduce
from itertools import product
from math import gcd
import numpy as np

class DomainWallSystem:
    def __init__(self, ferroelectric_structure, polarization_vector, domain_plane=(0, 0, 1), domain_angle=180, 
                 slab_size=1, stack_size=3, stack_axis=2, translation1=np.zeros(3), translation2=np.zeros(3), layer_distance=2):
        """
        Initialize the domain wall system with input parameters.
        """
        self.atoms = ferroelectric_structure
        self.P1 = polarization_vector
        self.domain_plane = np.array(domain_plane)
        self.domain_angle = np.radians(domain_angle)
        self.stack_axis = stack_axis
        self.translation1 = np.array(translation1)
        self.translation2 = np.array(translation2)
        self.layer_distance = layer_distance
        
        self.supercell_matrix = [slab_size] * 3
        self.supercell_matrix[self.stack_axis] = stack_size

    def find_minimal_orthogonal_vectors(self, c):
        """
        Find two minimal integer vectors orthogonal to the given vector.
        """
        def is_perpendicular(v1, v2):
            return np.dot(v1, v2) == 0
        def normalize_vector(v):
            g = reduce(gcd, v) if any(v) else 1
            return tuple(v // g)
        def vector_norm(v):
            return sum(abs(x) for x in v)
        c = np.array(c, dtype=int)
        candidates_a = []
        for x, y, z in product(range(-10, 11), repeat=3): 
            a = np.array([x, y, z])
            if np.all(a == 0): 
                continue
            if is_perpendicular(a, c):
                a = normalize_vector(a)
                candidates_a.append(a)
        candidates_a = sorted(set(candidates_a), key=vector_norm)
        a = candidates_a[0]
        candidates_b = []
        for x, y, z in product(range(-10, 11), repeat=3): 
            b = np.array([x, y, z])
            if np.all(b == 0) or np.array_equal(b, a):
                continue
            if is_perpendicular(b, c) and is_perpendicular(b, a):
                b = normalize_vector(b)
                candidates_b.append(b)
        candidates_b = sorted(set(candidates_b), key=vector_norm)
        b = candidates_b[0]
        return a, b

    def cut_and_rotate(self, atoms):
        """
        Cut and rotate the crystal to align with the specified domain plane.
        """
        a, b = self.find_minimal_orthogonal_vectors(self.domain_plane)
        slab = cut(atoms, a=a, b=b, c=self.domain_plane)
        rotate(slab, slab.cell[0], (1, 0, 0), slab.cell[1], (0, 1, 0))
        transformation_matrix_cut = np.array([a, b, self.domain_plane]).T
        P1_cut = np.linalg.inv(transformation_matrix_cut) @ self.P1
        new_basis = np.array([slab.cell[0], slab.cell[1], slab.cell[2]])
        target_basis = np.eye(3)
        rotation_matrix = np.linalg.inv(new_basis) @ target_basis
        self.P1 = rotation_matrix @ P1_cut
        return slab

    def calculate_rotation_matrix(self):
        """
        Calculate the rotation matrix for the specified domain angle.
        """
        P1_norm = self.P1 / np.linalg.norm(self.P1)
        if not np.isclose(P1_norm[0], 0):
            v = np.array([0, 1, 0]) 
        else:
            v = np.array([1, 0, 0])
        u = v - np.dot(v, P1_norm) * P1_norm
        u = u / np.linalg.norm(u)
        P2_norm = np.cos(self.domain_angle) * P1_norm + np.sin(self.domain_angle) * u
        dot_product = np.round(np.dot(P1_norm, P2_norm), 6)
        
        if np.isclose(dot_product, 1.0): 
            return np.eye(3) 
        elif np.isclose(dot_product, -1.0):
            if not np.isclose(P1_norm[0], 0):
                R = np.array([-P1_norm[1], P1_norm[0], 0])
            else:
                R = np.array([0, -P1_norm[2], P1_norm[1]])
        else:
            R = np.cross(P1_norm, P2_norm)
            
        R = R / np.linalg.norm(R)
        K = np.array([[0, -R[2], R[1]], [R[2], 0, -R[0]], [-R[1], R[0], 0]])
        I = np.eye(3)
        return I + np.sin(self.domain_angle) * K + (1 - np.cos(self.domain_angle)) * np.dot(K, K)

    def redifined_cell(self, atoms, new_cell):
        """
        Adjust the cell of an atom structure while preserving atomic positions.
        """
        atoms.set_cell(new_cell, scale_atoms=False)
        positions = atoms.get_positions()
        cell = atoms.get_cell()
        new_positions = np.dot(positions, np.linalg.inv(cell))
        atoms.set_positions(np.dot(new_positions, new_cell))
        atoms.center()
        return atoms

    def translate_atoms(self, atoms, translation):
        """
        Translate an Atoms object along a specified axis and apply periodic boundary conditions.
        """
        atoms.positions += translation
        atoms.wrap()
        return atoms

    def generate_domain_structure(self):
        """
        Generate a single-domain structure with a specified domain wall angle.
        """
        slab1 = self.cut_and_rotate(self.atoms.copy())
        slab1 = self.translate_atoms(slab1, self.translation1)
        slab1.wrap()
        slab1.center()
        
        slab2 = slab1.copy()
        rotation_matrix = self.calculate_rotation_matrix()
        slab2.positions = np.dot(slab2.positions, rotation_matrix.T)
        slab2.cell = np.dot(slab2.cell, rotation_matrix.T) 
        slab2 = self.redifined_cell(slab2, slab1.cell.copy())
        slab2 = self.translate_atoms(slab2, self.translation2)
        slab2.wrap()
        slab2.center()

        slab1 = slab1 * self.supercell_matrix
        slab2 = slab2 * self.supercell_matrix
        
        stacked_cell = slab1.cell.copy()
        stacked_cell[self.stack_axis, self.stack_axis] = slab1.cell[self.stack_axis, self.stack_axis] + slab2.cell[self.stack_axis, self.stack_axis] + self.layer_distance
        slab2.positions[:, self.stack_axis] += slab1.cell[self.stack_axis, self.stack_axis] + self.layer_distance
        stacked_positions = np.vstack([slab1.positions, slab2.positions])
        
        stacked_slab = Atoms(
            symbols=slab1.get_chemical_symbols() + slab2.get_chemical_symbols(),
            positions=stacked_positions,
            cell=stacked_cell,
            pbc=True
        )
        stacked_slab.wrap()
        stacked_slab.center()

        return slab1, slab2, stacked_slab

In [ ]:
atoms = read("../examples/Domain_walls/GeTe/CONTCAR")
slab1, slab2, stacked_slab = DomainWallSystem(atoms, [1, 1, 1], domain_plane=(0, 0, 1), domain_angle=180, slab_size=1, stack_size=3, stack_axis=2, translation1=np.array([0, 0, 0]), translation2=np.array([0, 0, 0]), layer_distance=0).generate_domain_structure()
view(stacked_slab)

In [ ]:
import numpy as np

def calculate_vector_sum_difference(atoms):
    positions = atoms.get_positions()
    symbols = atoms.get_chemical_symbols()
    cation_positions = np.array([positions[i] for i, symbol in enumerate(symbols) if symbol in ["Ge"]])
    anion_positions = np.array([positions[i] for i, symbol in enumerate(symbols) if symbol in ["Te"]])
    cation_sum = np.sum(cation_positions, axis=0)
    anion_sum = np.sum(anion_positions, axis=0)
    return cation_sum - anion_sum

def calculate_angle(vector1, vector2):
    v1 = np.array(vector1)
    v2 = np.array(vector2)
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)
    if magnitude_v1 == 0 or magnitude_v2 == 0:
        raise ValueError("One or both of the vectors have zero magnitude.")
    cos_theta = dot_product / (magnitude_v1 * magnitude_v2)
    cos_theta = np.clip(cos_theta, -1.0, 1.0)
    angle_radians = np.arccos(cos_theta)
    angle_degrees = np.degrees(angle_radians)
    return angle_degrees

vector1 = calculate_vector_sum_difference(slab1)
vector2 = calculate_vector_sum_difference(slab2)

angle = calculate_angle(vector1, vector2)
print(f"The angle between vector1 and vector2 is {angle:.2f} degrees.")

In [ ]:
calculate_vector_sum_difference(atoms)